<a href="https://colab.research.google.com/github/kmjohnson3/Intro-to-MRI/blob/master/NoteBooks/BasicWeightedImages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic MRI Contrast Images

This notebook will simulate image for Gradient Echo and Spin Echo sequences. It uses a digital phantom of the brain:

* http://www.bic.mni.mcgill.ca/brainweb/

* C.A. Cocosco, V. Kollokian, R.K.-S. Kwan, A.C. Evans : "BrainWeb: Online Interface to a 3D MRI Simulated Brain Database"
NeuroImage, vol.5, no.4, part 2/4, S425, 1997 -- Proceedings of 3-rd International Conference on Functional Mapping of the Human Brain, Copenhagen, May 1997.

* R.K.-S. Kwan, A.C. Evans, G.B. Pike : "MRI simulation-based evaluation of image-processing and classification methods"
IEEE Transactions on Medical Imaging. 18(11):1085-97, Nov 1999.
R.K.-S. Kwan, A.C. Evans, G.B. Pike : "An Extensible MRI Simulator for Post-Processing Evaluation"

* Visualization in Biomedical Computing (VBC'96). Lecture Notes in Computer Science, vol. 1131. Springer-Verlag, 1996. 135-140.

* D.L. Collins, A.P. Zijdenbos, V. Kollokian, J.G. Sled, N.J. Kabani, C.J. Holmes, A.C. Evans : "Design and Construction of a Realistic Digital Brain Phantom"
IEEE Transactions on Medical Imaging, vol.17, No.3, p.463--468, June 1998.

## Limitations

*   The $T1$ and $T2$ values are assumed to be the same for each tissue type.
*   The $T2'$ (and subsequently $T2^*$) are calculated from a simluation based on the segmentation of the anatomy and assumed magnetic suceptibility. It therfore, does not do a great job in replicating $T2^*$ seen in-vio.


In [ ]:
# For interactive plotting
from ipywidgets import interact, interactive, FloatSlider, IntSlider, ToggleButton
from IPython.display import clear_output, display, HTML

# General utilities
import numpy as np
import glob
import h5py

# for plotting modified style for better visualization
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['lines.linewidth'] = 4
mpl.rcParams['axes.titlesize'] = 16
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12
mpl.rcParams['legend.fontsize'] = 12


# For interactive plotting
from ipywidgets import interact, interactive, FloatSlider, IntSlider, ToggleButton
from IPython.display import clear_output, display, HTML

# for plotting modified style for better visualization
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['lines.linewidth'] = 4
mpl.rcParams['axes.titlesize'] = 16
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12
mpl.rcParams['legend.fontsize'] = 12


# Download the Digitial Phantom  
This will download a tar containing multiple HDF5 files. Each file represents a tissue with one additional file to provide T2' values.

In [ ]:
# Download a tar with tissues
!wget -O brain_sim.tar https://www.dropbox.com/scl/fi/uuf51rdqcb3ml44xt3uz5/brain_sim.tar?rlkey=t7plwqqr5i3qba66wbbw0i1z3&dl=0
!tar xvf brain_sim.tar

# Define a Tissue Class
Tissues are represented by a structure which contains its name, a volume, and relevant parameters. The volume defines a fuzzy segmentation of the tissue. It also defined loading and saving functions from the HDF5 file.

In [ ]:
class Tissue:
    '''
    Tissue class to store MRI data and parameters
    '''

    def __init__(self, volume):
        self.volume = volume
        self.T1 = -1
        self.T2 = -1
        self.T2star = -1
        self.density = -1
        self.susceptibility = -1
        self.name = 'None'
        self.freq = 0.0

    def __getitem__(self, idx):
        return ((self.volume[idx] + 128) / 255)

    def __setitem__(self, idx, value):
        self.volume[idx] = value

    def export(self, filename):

        with h5py.File(filename, 'w') as f:
            f.create_dataset('volume', data=self.volume, compression='gzip')
            f.attrs['T1'] = self.T1
            f.attrs['T2'] = self.T2
            f.attrs['T2star'] = self.T2star
            f.attrs['density'] = self.density
            f.attrs['susceptibility'] = self.susceptibility
            f.attrs['name'] = self.name
            f.attrs['freq'] = self.freq

    def load(self, filename):

        with h5py.File(filename, 'r') as f:
            self.volume = np.array(f['volume']).astype(np.float32)
            self.T1 = f.attrs['T1']
            self.T2 = f.attrs['T2']
            self.T2star = f.attrs['T2star']
            self.density = f.attrs['density']
            self.susceptibility = f.attrs['susceptibility']
            self.name = f.attrs['name']
            self.freq = f.attrs['freq']



# Load Tissues files and the gradient files

In [ ]:
files = glob.glob('subject54*.h5')
tissues = []
for file in files:
  tissue = Tissue(None)
  tissue.load(file)
  tissues.append(tissue)


with h5py.File('gradient.h5','r') as hf:
  grad = np.array(hf['gradient']).astype(np.float32)
grad /= np.max(grad)

# Define the simulation and plotting functions
This will define a function to calculate the signal from the volume and imaging parameters. The simulation is based on analytical solutions to be fast simulating:

* **Spoiled Gradient Echo** : This is based on a simple steady state solution for $M_z$ with $M_{xy}$ cacluated from $M_z$ and the gradient of the simulated field.

* **Spin echo** : This is a for a $90^\circ$ - $180^\circ$ sequence. The flip angle thus cannot be set by the user and will automatically be set to $90^\circ$.


In [ ]:
def calc_signal( TE, TR, B0, freq, alpha, M0, T1, T2, T2star, spin_echo, grad_slice):
    # T1, T2, T2star in ms
    # B0 in Hz
    # alpha in degrees
    # M0 in arbitrary units
    # TE, TR in ms

    # convert B0 to rad/s
    B0 = B0 * 2 * np.pi

    # convert alpha to rad
    alpha = alpha * np.pi / 180

    # calculate the longitudinal magnetization based on flip angle
    #Mxy = Mz * np.sin(alpha) * np.exp(-TE / T2star)
    if spin_echo:
        Mz180 = -M0*(1 - np.exp(-0.5*TE/T1))
        Mz = M0 + (Mz180 - M0) * np.exp(-(TR-0.5*TE) / T1)
        Mxy = Mz * np.exp(-TE / T2)
    else:
        Mz = M0 * (1 - np.exp(-TR / T1)) / (1 - np.cos(alpha) * np.exp(-TR / T1))
        Mxy = Mz * np.sin(alpha) * np.exp(-TE / T2star) * np.exp(-20*grad_slice*TE) * np.exp(-1j*2*np.pi*freq*TE/1000)

    return Mxy


def plot_image(TR, TE, flip, slice, spin_echo, noise_level):


    #print(f'{TR}, {TE}, {flip}')

    signal = np.zeros((434, 362), dtype=np.complex128)
    for tissue in tissues:
            signal += calc_signal(TE, TR, 0, tissue.freq, flip, tissue[slice]*tissue.density, tissue.T1, tissue.T2, tissue.T2star, spin_echo, grad[slice])

    signal = np.abs( signal + np.random.normal(0, noise_level, signal.shape))

    plt.figure()
    plt.imshow(np.flip(signal), cmap='gray')

    #plt.imshow(np.flip(phs_tissue_simulated[slice]), cmap='gray')

    plt.xticks([])
    plt.yticks([])

    if spin_echo:
        plt.title(f' SE:flip={int(flip)},TR={int(TR)},TE={int(TE)} ')
    else:
        plt.title(f'GRE:flip={int(flip)},TR={int(TR)},TE={int(TE)} ')

    plt.colorbar()
    plt.show()



TRslider = FloatSlider(min=2, max=5000, step=1, value=50, description='TR [ms]',continuous_update=True)
TEslider = FloatSlider(min=1, max=50, step=1, value=3, description='TE [ms]',continuous_update=True)
flip_slider = FloatSlider(min=1, max=90, step=1, value=10, description='Flip [deg.]',continuous_update=True)
spin_echo_toggle = ToggleButton(value=False, description='Toggle Spin echo', continuous_update=True)

def update_max_TE(change):
    TEslider.max = min(change['new']-1,100.0)

def update_flip_min(change):

    if change['new']:
        flip_slider.max = 90
        flip_slider.min = 90
    else:
        flip_slider.max = 90
        flip_slider.min = 1


TRslider.observe(update_max_TE, names='value')
spin_echo_toggle.observe(update_flip_min, names='value')

# Main simulation
Things to try:

1. Set the flip angle to 90 and the TE to 1. Sweep the TR. Does the contrast behave as expected? What type of weighting might this be?

1. Set the flip angle to 90 and TR to 5000. Sweep the TE. Does the contrast behave as expected? What type of weighting might this be?

1. Try 2. Again but swap between spin and gradient echo images. Look at multiple slices to see differences.

1. For gradient echo, Set the TR to 10 and TE to 1. Sweep the flip angle. Does the contrast behave as expected? What type of weighting might this be?

1. Mix the changes in TE, TR, and flip angle.




In [ ]:
w = interactive(plot_image,
                TR=TRslider,
                TE=TEslider,
                flip=flip_slider,
                slice=IntSlider(min=0, max=362, step=1, value=130, description='Slice',continuous_update=True),
                spin_echo=spin_echo_toggle,
                noise_level=FloatSlider(min=0, max=0.1, step=0.001, value=0.01, description='Noise level',continuous_update=True)
                )
display(w)